<a href="https://colab.research.google.com/github/Hurri301/Advanced-Computational-Techniques-for-Big-Imaging-and-Signal-Data/blob/main/Semantic_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Define the paths
image_dir = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/images'
gt_dir = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/targets'
train_list = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/train.txt'
val_list = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/validation.txt'
test_list = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/test.txt'


In [3]:
# Inspect unique values in ground truth masks
import cv2
import numpy as np

# Define paths (example for one image)
gt_path = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/targets/sample.png'

gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
unique_values = np.unique(gt)
print("Unique values in the ground truth mask:", unique_values)

Unique values in the ground truth mask: [None]


In [11]:
import os
import cv2
import numpy as np
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

class MyDataset(Dataset):
    def __init__(self, image_dir, gt_dir, filelist, transform=None):
        with open(filelist, 'r') as f:
            self.fns = f.read().splitlines()
        self.image_dir = image_dir
        self.gt_dir = gt_dir
        self.transform = transform

    def __len__(self):
        return len(self.fns)

    def __getitem__(self, idx):
        cur_fn = self.fns[idx]
        img_path = os.path.join(self.image_dir, cur_fn + '.jpg')
        gt_path = os.path.join(self.gt_dir, cur_fn + '.png')

        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"Image file not found: {img_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
        if gt is None:
            raise FileNotFoundError(f"Ground truth file not found: {gt_path}")

        # Ensure the mask is a NumPy array
        gt = np.array(gt)

        # Debugging: Print unique values in the mask
        unique_values = np.unique(gt)
        print(f"Unique values in the ground truth mask for {cur_fn}: {unique_values}")

        # Clip the ground truth mask to be within the valid range
        gt = np.clip(gt, 0, 20)

        if self.transform:
            augmented = self.transform(image=img, mask=gt)
            img = augmented['image']
            gt = augmented['mask']

        return img, gt

# Define your transformations
train_transform = A.Compose([
    A.RandomCrop(64, 64),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(64, 64),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Define the paths
image_dir = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/images'
gt_dir = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/targets'
train_list = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/train.txt'
val_list = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/validation.txt'
test_list = '/content/drive/MyDrive/VOC_exercise/VOC_exercise/test.txt'

# Create DataLoaders
def create_dataloader(image_dir, gt_dir, filelist, transform, batch_size=16, shuffle=True):
    dataset = MyDataset(image_dir, gt_dir, filelist, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

train_loader = create_dataloader(image_dir, gt_dir, train_list, train_transform, batch_size=16, shuffle=True)
val_loader = create_dataloader(image_dir, gt_dir, val_list, val_transform, batch_size=16, shuffle=False)
test_loader = create_dataloader(image_dir, gt_dir, test_list, val_transform, batch_size=16, shuffle=False)


In [12]:
import torch.optim as optim
import torch.nn as nn

# Initialize model, loss function, and optimizer
model = SimpleCNN(num_classes=21)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, masks in train_loader:
        # Move images and masks to the device (GPU or CPU)
        images = images.float()  # Ensure the images are in float
        masks = masks.long()     # Ensure the masks are in long format

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Streaming output truncated to the last 5000 lines.
Unique values in the ground truth mask for 2009_003783: [ 0 19]
Unique values in the ground truth mask for 2011_002561: [  0 112 147]
Unique values in the ground truth mask for 2009_001871: [  0  57 112 132]
Unique values in the ground truth mask for 2008_001610: [ 0 19]
Unique values in the ground truth mask for 2008_001926: [  0 150]
Unique values in the ground truth mask for 2007_007902: [  0 108 147]
Unique values in the ground truth mask for 2009_004519: [ 0 52]
Unique values in the ground truth mask for 2007_006066: [  0  52  57 112]
Unique values in the ground truth mask for 2011_000278: [  0  14 128 147]
Unique values in the ground truth mask for 2008_005839: [ 0 75]
Unique values in the ground truth mask for 2010_002527: [  0  33 112]
Unique values in the ground truth mask for 2007_000645: [  0 113]
Unique values in the ground truth mask for 2008_000259: [  0 147]
Unique values in the ground truth mask for 2011_002410: [  0 12

Unique values in the ground truth mask: [None]
